In [1]:
deps_path = '/kaggle/input/czii-cryoet-dependencies'

In [2]:
! cp -r /kaggle/input/czii-cryoet-dependencies/asciitree-0.3.3/ asciitree-0.3.3/

In [3]:
! pip wheel asciitree-0.3.3/asciitree-0.3.3/
! cp /kaggle/input/best-weights-code/* ./ -r
! cp /kaggle/input/best-weights ./ -r
! cp /kaggle/input/voxhrnet-v2/voxhrnetV2.py ./ -r
! cp /kaggle/input/load-vox-net/load_model.py ./ -r
! cp /kaggle/input/hrnet-v1/* ./ -r

Processing ./asciitree-0.3.3/asciitree-0.3.3
  Preparing metadata (setup.py) ... - done
  Created wheel for asciitree: filename=asciitree-0.3.3-py3-none-any.whl size=5034 sha256=9187800ad11e4c3249fbd9ab598786d547af0ff351558238d1b040560ceae9af
  Stored in directory: /root/.cache/pip/wheels/77/df/a8/c6318c87827b53ea48635bf95b1249186d3d21f041176e062e
Successfully built asciitree


In [4]:
! cp /kaggle/input/d/luoziqian/unet2e3d-6c/* ./
! cp /kaggle/input/voxhrnet/voxhrnet.py ./ -r
! cp /kaggle/input/vox-networks-dataset/voxhrnet.py ./
! cp /kaggle/input/vox-networks-dataset/config_small.yaml ./
! cp /kaggle/input/vox-networks-dataset/model10.py ./
! cp /kaggle/input/voxresnet-v0/voxresnetV0.py ./

In [5]:
!pip install asciitree-0.3.3-py3-none-any.whl
! pip install /kaggle/input/einops-0-8-none-any/einops-0.8.0-py3-none-any.whl

Processing ./asciitree-0.3.3-py3-none-any.whl
Processing /kaggle/input/einops-0-8-none-any/einops-0.8.0-py3-none-any.whl


In [6]:
! pip install -q --no-index --find-links {deps_path} --requirement {deps_path}/requirements.txt

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.3 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
dask-expr 1.1.15 requires dask==2024.9.1, but you have dask 2024.11.0 which is incompatible.
kfp 2.5.0 requires google-cloud-storage<3,>=2.2.1, but you have google-cloud-storage 1.44.0 which is incompatible.
kfp 2.5.0 requires requests-toolbelt<1,>=0.8.0, but you have requests-toolbelt 1.0.0 which is incompatible.
rapids-dask-dependency 24.8.0a0 requires dask==2024.7.1, but you have dask 2024.11.0 which is incompatible.
rapids-dask-dependency 24.8.0a0 requires distributed==2024.7.1, but you have distributed 2024.11.0 which is incompatible.


In [7]:
! pip install /kaggle/input/vox-networks-dataset/yacs-0.1.8-py3-none-any.whl

Processing /kaggle/input/vox-networks-dataset/yacs-0.1.8-py3-none-any.whl


In [8]:
from typing import List, Tuple, Union
import numpy as np
import torch
from monai.data import DataLoader, Dataset, CacheDataset, decollate_batch
from monai.transforms import (
    Compose, 
    EnsureChannelFirstd, 
    Orientationd,  
    AsDiscrete,  
    RandFlipd, 
    RandRotate90d, 
    NormalizeIntensityd,
    RandCropByLabelClassesd,
)

In [9]:
import os
import shutil
import tempfile
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from monai.transforms import (
    Compose,
    Orientationd,
    RandFlipd,
    RandShiftIntensityd,
    RandRotate90d,
)
from monai.data import (
    ThreadDataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    set_track_meta,
)
from monai.inferers import sliding_window_inference
from monai.networks.nets import SwinUNETR
from monai.metrics import DiceMetric
from monai.losses import DiceCELoss
import torch
import einops
import warnings


warnings.filterwarnings("ignore")
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Define some helper functions


### Patching helper functions

These are mostly used to split large volumes into smaller ones and stitch them back together. 

In [10]:
def calculate_patch_starts_with_overlap(
    dimension_size: int, patch_size: int, overlap: int
) -> List[int]:
    if dimension_size <= patch_size:
        return [0]

    num_patches = np.ceil(
        (dimension_size - overlap) / (patch_size - overlap) + 1
    ).astype(int)
    patch_starts = []
    for i in range(num_patches):
        pos = int(i * (patch_size - overlap))
        if pos + patch_size > dimension_size:
            pos = dimension_size - patch_size
        if pos not in patch_starts:
            patch_starts.append(pos)
    return patch_starts


def extract_3d_patches_overlap(
    arrays: List[np.ndarray],
    patch_sizes: Tuple[int, int, int],
    overlap_sizes: Tuple[int, int, int],
) -> Tuple[List[np.ndarray], List[Tuple[int, int, int]]]:

    patch_starts_x = calculate_patch_starts_with_overlap(
        arrays[0].shape[0], patch_sizes[0], overlap_sizes[0]
    )
    patch_starts_y = calculate_patch_starts_with_overlap(
        arrays[0].shape[1], patch_sizes[1], overlap_sizes[1]
    )
    patch_starts_z = calculate_patch_starts_with_overlap(
        arrays[0].shape[2], patch_sizes[2], overlap_sizes[2]
    )
    patch_size_d, patch_size_h, patch_size_w = patch_sizes
    patches = []
    coordinates = []
    for arr in arrays:
        for x in patch_starts_x:
            for y in patch_starts_y:
                for z in patch_starts_z:
                    patch = arr[
                        x : x + patch_size_d, y : y + patch_size_h, z : z + patch_size_w
                    ]
                    patches.append(patch)
                    coordinates.append((x, y, z))

    return patches, coordinates

## Reading in the data

In [11]:
TRAIN_DATA_DIR = "/kaggle/input/create-numpy-dataset-exp-name"
TEST_DATA_DIR = "/kaggle/input/czii-cryo-et-object-identification"

## Initialize the model

This model is pretty much directly copied from [3D U-Net PyTorch Lightning distributed training](https://www.kaggle.com/code/zhuowenzhao11/3d-u-net-pytorch-lightning-distributed-training)

In [12]:
from torch.nn.modules import Module
from torch import nn

In [13]:
!cp -r /kaggle/input/unet2d3e/* ./

In [14]:
import lightning.pytorch as pl

from monai.networks.nets import UNet
from monai.losses import TverskyLoss
from monai.metrics import DiceMetric

# use warmup lr scheduler
from torch.optim.lr_scheduler import (
    CosineAnnealingLR,
    CosineAnnealingWarmRestarts,
    StepLR,
)
import lightning.pytorch as pl

from monai.networks.nets import UNet, AttentionUnet
from monai.losses import TverskyLoss
from monai.metrics import DiceMetric

# use warmup lr scheduler
from torch.optim.lr_scheduler import (
    CosineAnnealingLR,
    CosineAnnealingWarmRestarts,
    StepLR,
)

In [15]:
config_v2 = {
    "MODEL": {
        "NAME": "voxhrnet",
        "EXTRA": {
            "STAGE2": {
                "NUM_MODULES": 1,
                "NUM_BRANCHES": 2,
                "BLOCK": "BASIC",
                "NUM_BLOCKS": [3, 3],
                "NUM_CHANNELS": [16, 32],
            },
            "STAGE3": {
                "NUM_MODULES": 1,
                "NUM_BRANCHES": 3,
                "BLOCK": "BASIC",
                "NUM_BLOCKS": [3, 3, 3],
                "NUM_CHANNELS": [16, 32, 64],
            },
        },
    }
}

In [16]:
from monai.networks.nets import UNet,SegResNet,DynUNet
from custom_vnet import CustomVNet
from model2_6c import Net
# from voxhrnet import build_model
from voxhrnetV2 import build_model as build_model_v2
from load_model import build_model as build_model_v3
from model10_v1 import build_model as build_model_v4
from voxresnetV0 import VoxResNet as VoxResNetV0

basic_unet = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=7,
    channels=(48, 64, 80, 80),
    strides=(2, 2, 1),
    num_res_units=1,
)
basic_unet_1 = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=7,
    channels=(48, 64, 80, 80),
    strides=(2, 2, 1),
    num_res_units=2,
)
basic_unet_6c = UNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=6,
    channels=(48, 64, 80, 80),
    strides=(2, 2, 1),
    num_res_units=1,
)

segresnet_6c_v1 = SegResNet(
    in_channels=1,
    out_channels=6,
    dropout_prob=0.1,
    upsample_mode="deconv",
)

basic_unet2e3d = Net(
    out_channels=7,
    arch="resnet18d",
    decoder_dim=[80, 80, 64, 32, 16],
    pretrained=False,
)

basic_dynunet_v1 = DynUNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=6,
    kernel_size=(3, 3, 3, 3),
    strides=((1, 1, 1), 2, 2, 1),
    upsample_kernel_size=(2, 2, 1),
    filters=[16, 24, 48, 80, 80],
    norm_name="instance",
    act_name="PRELU",
    deep_supervision=True,
)

basic_dynunet_v1 = DynUNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=6,
    kernel_size=(3, 3, 3, 3),
    strides=((1, 1, 1), 2, 2, 1),
    upsample_kernel_size=(2, 2, 1),
    filters=[16, 24, 48, 80, 80],
    norm_name="instance",
    act_name="PRELU",
    deep_supervision=True,
)

basic_dynunet_v2 = DynUNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=6,
    kernel_size=(3, 3, 3, 3, 3),
    strides=((1, 1, 1), 2, 2, 2, 1),
    upsample_kernel_size=(2, 2, 2, 1),
    filters=[16, 24, 48, 80, 80],
    norm_name="instance",
    act_name="PRELU",
    deep_supervision=True,
)

# basic_voxhrnet_v0 = build_model(1,6)
basic_voxhrnet_v2 = build_model_v2(1,6,config_v2)
basic_voxhrnet_v3 = build_model_v3()
bisic_voxhrnet_v4 = build_model_v4()

basic_voxresnet_v0 = VoxResNetV0(in_channels=1, n_classes=7)

basic_dense_vnet = CustomVNet(in_channels=1, classes=7)

In [17]:
import os

best_weights_dir = "best-weights"
os.listdir(best_weights_dir)
# best_weights_path_list = [
#  'best-weights/epoch122-step2952-valid_loss0.3625-val_metric0.8367.ckpt',
#  'best-weights/epoch148-step3576-valid_loss1.1154-val_metric0.7722.ckpt',
#  'best-weights/epoch153-step3696-valid_loss0.3021-val_metric0.8900.ckpt',
#  'best-weights/epoch194-step4680-valid_loss1.0213-val_metric0.8788.ckpt',
#  '/kaggle/input/voxresnet-v0/epoch195-step4704-valid_loss0.4258-val_metric0.7914.ckpt',
# ] \
#  + ['/kaggle/input/hrnet-v1/epoch188-step4536-valid_loss0.4231-val_metric0.8659.ckpt'] \
#  + ['/kaggle/input/unet2e3d-6c/pytorch/default/1/unet2E3D-v1-epoch114-val_loss0.55-val_metric0.53-step2760.ckpt'] \
#  + ['/kaggle/input/my-weights/unet3D-epoch173-val_loss0.53-val_metric0.54-step4176.ckpt']  \
#  + ['best-weights/epoch152-step3672-valid_loss0.4333-val_metric0.7929.ckpt'] \
#  +['/kaggle/input/segresnet-6c/pytorch/default/1/epoch314-val_loss0.54-val_metric0.54-step7560.ckpt']

#  # + ['/kaggle/input/hrnet-v1/epoch188-step4536-valid_loss0.4231-val_metric0.8659.ckpt'] 


 # +['/kaggle/input/my-weights/unet3D-epoch173-val_loss0.53-val_metric0.54-step4176.ckpt']



 # +['/kaggle/input/segresnet-6c/pytorch/default/1/epoch314-val_loss0.54-val_metric0.54-step7560.ckpt']

#  + ['/kaggle/input/dynunet-6c/pytorch/default/1/DynUnet-epoch189-val_loss0.51-val_metric0.55-step4560.ckpt'] \
#  +['/kaggle/input/segresnet-6c/pytorch/default/1/epoch314-val_loss0.54-val_metric0.54-step7560.ckpt']

#  +['/kaggle/input/my-weights/unet3D-epoch173-val_loss0.53-val_metric0.54-step4176.ckpt']
 # +['/kaggle/input/unet2e3d-6c/pytorch/default/1/unet2E3D-v1-epoch114-val_loss0.55-val_metric0.53-step2760.ckpt'] \
# best_weights_path_list=['/kaggle/input/dynunet-6c/pytorch/default/1/DynUnet-epoch189-val_loss0.51-val_metric0.55-step4560.ckpt']
# best_weights_path_list=['/kaggle/input/dynunet-6c/pytorch/default/2/DynUnet-v2-epoch164-val_loss0.53-val_metric0.53-step3960.ckpt']
# best_weights_path_list=['/kaggle/input/vox-networks-dataset/epoch133-step3216-valid_loss0.4223-val_metric0.8612.ckpt']
best_weights_path_list = ['/kaggle/input/unet2e3d-7c/pytorch/default/1/unet2e3d-epoch135-step3264-valid_loss0.4315-val_metric0.7775.ckpt']

In [18]:
import copy
import torch
dummy_input = torch.rand((1, 1, 128, 384, 384)).half().cuda()

model_list = []
for idx,path in enumerate(best_weights_path_list):
    ckpt = torch.load(path)
    state_dict_ = ckpt["state_dict"]
    state_dict = {}
    for k in state_dict_.keys():
        if "model." in k:
            state_dict[k[6:]] = state_dict_[k]
    try:
        model = copy.deepcopy(basic_unet)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load unet")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_unet.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        pass
    try:
        model = copy.deepcopy(basic_unet_1)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load unet_1")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_unet_1.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        # print("load failed")
        pass
    try:
        model = copy.deepcopy(basic_dense_vnet)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load vnet")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_vnet.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        # print("load failed")
        pass
    try:
        model = copy.deepcopy(basic_unet_6c)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load unet_6c")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_unet_6c.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        # print("load failed")
        pass
    try:
        model = copy.deepcopy(basic_unet2e3d)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load unet2e3d")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_unet2e3d.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        # print("load failed")
        pass
    try:
        model = copy.deepcopy(segresnet_6c_v1)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load segresnet_6c_v1")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_segresnet_6c_v1.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        # print("load failed")
        pass
    try:
        model = copy.deepcopy(basic_dynunet_v1)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load dynunet")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_dynunet.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        # print("load failed")
        pass
    try:
        model = copy.deepcopy(basic_dynunet_v2)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load basic_dynunet_v2")  
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_basic_dynunet_v2.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        # print("load failed")
        pass
    try:
        model = copy.deepcopy(basic_voxhrnet_v0)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load basic_voxhrnet_v0")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_basic_voxhrnet_v0.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        pass
    try:
        model = copy.deepcopy(basic_voxhrnet_v3)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load basic_voxhrnet_v3")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_basic_voxhrnet_v3.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        pass
    try:
        model = copy.deepcopy(bisic_voxhrnet_v4)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load bisic_voxhrnet_v4")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_bisic_voxhrnet_v4.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        pass
    try:
        model = copy.deepcopy(basic_voxhrnet_v2)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load basic_voxhrnet_v2")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_basic_voxhrnet_v2.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        pass
    try:
        model = copy.deepcopy(basic_voxresnet_v0)
        model.load_state_dict(state_dict)
        model_list.append(model.to("cpu"))
        print("load basic_voxresnet_v0")
        model=model.eval().half().cuda()
        dynamic_axes = {"images": {0: "batch"}, "output": {0: "batch"}}
        output_name=f"{idx}_basic_voxresnet_v0.onnx"
        torch.onnx.export(
            model,
            dummy_input,
            output_name,
            verbose=False,
            input_names=['images'],
            output_names=['output'],
            dynamic_axes=dynamic_axes,
            do_constant_folding=True,
            opset_version=12        
        )
        continue
    except:
        pass
    

load unet2e3d
